<a href="https://colab.research.google.com/github/MaheepChaudhary/GANs_mnist/blob/main/GANs_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing the necessary packages
import tensorflow as tf
import tensorflow.keras as tk
from tensorflow.keras.datasets import mnist 
import imageio
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.layers import Dense,Conv2DTranspose,Conv2D,Dropout,Flatten,MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import activations
import matplotlib.pyplot as plt

In [2]:
#preprocessing of dataset(normalization and shuffling of images)
batch_size = 256

dataset = mnist.load_data(path="mnist.npz")
x_train,y_train,x_test,y_test = dataset[0][0],dataset[0][1],dataset[1][0],dataset[1][1]
x_train = x_train/255.0
x_train = shuffle(x_train)
x_train = tf.data.Dataset.from_tensor_slices(x_train).batch(batch_size)

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
#generator model
def generator_model():
  model = Sequential()
  model.add(Dense(7*7*256))
  model.add(Dropout(0.2))
  model.add(tf.keras.layers.Reshape((7,7,256)))
  model.add(Conv2DTranspose(128,kernel_size=(5,5),activations = 'relu'))
  model.add(Dropout(0.2))
  model.add(Conv2DTranspose(64,kernel_size=(3,3),activations = 'relu'))
  model.add(Dropout(0.2))
  model.add(Conv2DTranspose(1,activations = 'relu'))
  return model

generator = generator_model()  

TypeError: ignored

In [ ]:
#discriminator model
def discriminator_model():
  model = Sequential()  
  model.add(Conv2D(64,kernel = (3,3),activation='relu'))
  model.add(Dropout(0.2))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(128,kernel = (3,3),activation='relu'))
  model.add(Dropout(0.3))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(256,kernel = (3,3),activation='relu'))
  model.add(Dropout(0.2))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(200,activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(100,activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(50,activation='relu'))
  model.add(Dense(1,activation='relu'))
  return model

In [ ]:
#optimizers
gen_optimizer = tf.keras.optimizers.Adam(1e-4)
dis_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
#generative_loss
def gen_loss(fake):
  return tk.losses.binary_crossentropy(tf.zeros_like(fake),fake,from_logits = True)

In [ ]:
#discriminator_loss
def dis_loss(fake,original):
  real_loss = tk.losses.binary_crossentropy(tf.ones_like(original),original,from_logits = True)
  fake_loss = tk.losses.binary_crossentropy(tf.zeros_like(fake),fake,from_logits = True)  
  total_loss= real_loss + fake_loss
  return total_loss  

In [ ]:
#training_Step
no_of_examples = 16
noise_dim = 100

def train_step(images,batch_size):
  t = batch_size
  noise = tf.random.normal([batch_size,noise_dim])
  
  with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
    gen_images = generator(noise,training = True)
    original = discriminator(images,training = True) 
    fake = discriminator(gen_images,training = True)
    generator_loss,discriminator_loss = gen_loss(fake),dis_loss(fake,original)
    gen_gradient,dis_gradient = gen_tape.gradient(generator_loss,generator.trainable_variables),gen_tape.gradient(discriminator_loss,discriminator.trainable_variables)

  gen_optimizer(gen_gradient,generator.trainable_variables)
  dis_optimizer(dis_gradient,discriminator.trainable_variables)  

In [ ]:
#training_function
epochs = 50
batch_size = 256

def train(dataset,epochs,batch_size):
  for i in range(epochs):
    for batch in dataset:
      train_step(batch,batch_size)  

    display.clear_output(wait = True)
    noise = tf.random.normal([no_of_examples,noise_dim])
    generate_and_save_image(generator,epoch+1,noise)

  display.clear_output(wait = True)
  generate_and_save_image(generator,epochs,noise)
  print("The number of epoch is ",epoch)


In [ ]:
#plotting and saving image function
def generate_and_save_image(model,epoch,noise):
  images = generator(noise,trainable = False)
  fig = plt.figure(figsize = (4,4))
  for i in range(len(images)):
    plt.subplot(4,4,i+1)
    plt.imshow(images[i])
  plt.show()
  plt.savefig('/content/images/epoch'+str(epoch)+'.png') 
   

In [ ]:
#training occurs
train(dataset,epochs,batch_size)